In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd

In [4]:
data =pd.read_csv('xauusd_M15.csv')
data

,time,open,high,low,close,tick_volume
0,2016-12-15 02:00:00,1142.34,1142.50,1142.14,1142.24,196
1,2016-12-15 02:15:00,1142.24,1143.23,1142.06,1142.63,1929
2,2016-12-15 02:30:00,1142.64,1144.01,1142.62,1143.34,4677
3,2016-12-15 02:45:00,1143.34,1144.47,1142.96,1143.62,3059
4,2016-12-15 03:00:00,1143.58,1143.78,1142.84,1142.89,2773
...,...,...,...,...,...,...
149812,2023-06-09 19:30:00,1960.87,1961.09,1960.25,1960.65,650
149813,2023-06-09 19:45:00,1960.65,1960.68,1959.68,1960.17,1032
149814,2023-06-09 20:00:00,1960.18,1960.64,1960.09,1960.24,282
149815,2023-06-09 20:15:00,1960.24,1960.93,1960.23,1960.85,156


In [ ]:

# Step 1: Prepare the Data

# Splitting data into input features and supply/demand zone information
features = ['Opening', 'High', 'Low', 'RSI', 'SMA']
zone_info = ['ZonePresence', 'ZoneLevel']

# Splitting the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2)

# Step 2: Data Preprocessing

# Normalize the input features using Min-Max scaler
scaler = MinMaxScaler()
train_data[features] = scaler.fit_transform(train_data[features])
test_data[features] = scaler.transform(test_data[features])

# Convert the zone information into numerical values if necessary
train_data['ZonePresence'] = train_data['ZonePresence'].map({'no zone': 0, 'zone': 1})
test_data['ZonePresence'] = test_data['ZonePresence'].map({'no zone': 0, 'zone': 1})


In [ ]:

# Convert the data into sequences of fixed length
sequence_length = 10

def create_sequences(data):
    X = []
    y = []
    for i in range(len(data) - sequence_length):
        X.append(data[features].values[i:i+sequence_length])
        y.append(data[zone_info].values[i+sequence_length])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_data)
X_test, y_test = create_sequences(test_data)

# Convert NumPy arrays to PyTorch tensors
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()


In [ ]:

# Step 3: Build the LSTM Model

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

input_size = len(features)
hidden_size = 64
output_size = len(zone_info)

model = LSTMModel(input_size, hidden_size, output_size)


In [ ]:

# Define hyperparameters
learning_rate = 0.001
batch_size = 32
num_epochs = 50

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Create a custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

# Create DataLoader objects for training and testing
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Training loop
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

    # Print the loss after every epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.6f}")

# Step 5: Evaluation and Prediction


In [ ]:

# Evaluate the model on the testing set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    predicted_zones = (torch.sigmoid(test_outputs) > 0.5).float()

# Calculate accuracy or other evaluation metrics
accuracy = (predicted_zones == y_test).sum().item() / (y_test.numel())
print(f"Accuracy: {accuracy * 100:.2f}%")

# Make predictions on unseen data
unseen_data = ...  # unseen data to predict on
unseen_data[features] = scaler.transform(unseen_data[features])
X_unseen, _ = create_sequences(unseen_data)
X_unseen = torch.from_numpy(X_unseen).float()

model.eval()
with torch.no_grad():
    unseen_outputs = model(X_unseen)
    unseen_predicted_zones = (torch.sigmoid(unseen_outputs) > 0.5).float()
